In [1]:
#   % run all_utilities.ipynb

In [2]:
def unique_counts(col_names, mobile_data):
    for count in range(len(col_names)):   
        list_names[count] = []
        list_names[count] = mobile_data[col_names[count]].unique()
        num_unique = mobile_data[col_names[count]].nunique(dropna = True)
        print("\nThere are " + str(num_unique) + " unique " + str(col_names[count]) + "s.")
        all_uniques.append(list_names[count])
    return (all_uniques)

In [3]:
def replace_copies(df, replace_this, replace_with):
    '''
    goes through raw_data and replaces specific strings with another specific string
    '''
    new_data = pd.DataFrame()
    for count in range(len(df)):
        for num in range(len(replace_this)):
            if df['name'][count] == replace_this[num]:
                #print("repeat")
                df = df.replace(to_replace = replace_this[num], 
                    value = replace_with[num]) 
    df = df.groupby(['name']).sum()
    df = df.reset_index()
    return df

In [4]:
def normalize_data(raw_data):
    '''
    Normalizes raw data based off the min/max
    '''
    x = raw_data[['values']].values.astype(float)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    return(x_scaled)

In [5]:
def apply_normalizer(normalized_data, device_type, device_list):
    '''
    Adds normalized data to dataframe 
    '''
    normalized_df = pd.DataFrame(normalized_data)
    normalized_df.columns = ['normalized']
    for count in range(len(normalized_df)):
         device_list.append(device_type)
    normalized_df['device'] = device_list        
    return normalized_df

In [6]:
def get_top_users(df, NUM_TOP):
    '''
    Takes the number of top users (NUM_TOP) and return dataframe with usernames and value counts
    '''
    user_counts = pd.DataFrame()
    user_counts  = df['username'].value_counts()
    user_counts = user_counts.reset_index()
    user_counts.columns = ['username', 'no. clicks']
    top_users = pd.DataFrame()
    for count in range(NUM_TOP):
        top_users[count] = user_counts.loc[count]
    top_users = top_users.T
#    top_users = top_users.drop(index = 3)
    return(top_users.reset_index())

In [7]:
def normalize_data_users(raw_data):
    '''
    Normalizes data based on min/max of indiviudal USER not total
    '''
    x_scaled = raw_data.assign(values=raw_data.groupby('username').transform(lambda x: (x - x.min()) / (x.max()- x.min())))
    return (x_scaled)